In [9]:
from ultralytics import YOLO
import torch
from PIL import Image
import cv2
import time
import sys
import os
from datetime import datetime

In [10]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\Aizat/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-5 Python-3.9.16 torch-1.13.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [23]:
#Custom Model

model = torch.hub.load('ultralytics/yolov5', 'custom', path='konlovephone.pt')
#model = torch.hub.load('ultralytics/yolov5', 'custom', path='gesture3.pt')

Using cache found in C:\Users\Aizat/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-5 Python-3.9.16 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [29]:
confi_threshold = 0.50 #threshold value for confidence score. if the gesture exceeds this value, it is accepted as true
gesturePresent = False;
startTime = 0;
elapseTime = 0;
delayDetection = 3; #how long to hold the gesture for
scanning_class = -1;

display_msg = '';

facePresent = False;
startTime_faceDet = 0;
elapseTime_faceDet = 0;
delayDetection_face = 6.0; #how long to hold the face for


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
masterPassword = [1,1,0]; #master password refers to the the class numbers in order
print(masterPassword) #printers the master password


#class number classification
#kon - 0
#minilove - 1
#telephone - 2



[1, 1, 0]


In [30]:
cap = cv2.VideoCapture(0)

cv2.namedWindow("img", cv2.WINDOW_NORMAL)
  
# Using resizeWindow()
cv2.resizeWindow("img", 900, 700)
password = [];



while True:
    
    #reads the frames
    _, img = cap.read()
    
    
    #convert to grayscaleq
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect the faces
    faces = face_classifier.detectMultiScale(gray, 1.1, 4)
    
    #If a new Face is Present
    if len(faces) == 1 and (startTime_faceDet == 0):
        facePresent = True;
        startTime_faceDet = time.time();
    
    #if only 1 face is present, checks if the holding time is enough
    if len(faces) == 1 and (startTime_faceDet > 0):
        elapseTime_faceDet = round((time.time() - startTime_faceDet), 2);
        if(elapseTime_faceDet > delayDetection_face):
            print("Face is Detected")
            break;
    
    #places the text
    pic = cv2.putText(img, str(elapseTime_faceDet), (5,50), cv2.FONT_HERSHEY_PLAIN, 5 ,(0, 0, 255));
    
    #if no face is detected or more than 1 is detected, reset the timer
    if (len(faces) == 0 or len(faces) > 1):
        facePresent = False;
        startTime_faceDet = 0;
        print('No Face Detected; Timer Reset');
        pic = cv2.putText(img, "No Face Detected", (5,60), cv2.FONT_HERSHEY_PLAIN, 5 ,(0, 0, 255));
    
    
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        pic = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Display
    cv2.imshow('img', pic)
    
    # Stop if escape key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release();
        cv2.destroyAllWindows();
        
        
while True:
    _, frames = cap.read()  #reads each frame
    results = model(frames) #has the model predict the frame and predicts for objects 
    
    #debug
    #print('---------------------')
    #print(results.pandas().xyxy[0])
    #print('Detected Gestures : ' + str(len(results.pandas().xyxy[0])))
    
    max_detected_confi = 0;
    
    #for each object detected 
    for i in range(0, len(results.pandas().xyxy[0])):
        confi = results.xyxy[0][i][4].numpy() #retrieves the confidence score
        #print(results.pandas().xyxy[0]);
        
        #takes the highest confidence score gesture. Instead of ensuring only 1 gesture is present. We just take the highest score present in the list
        if confi > max_detected_confi:
            max_detected_confi = confi
            x0,y0,x1,y1,confi,cla = results.xyxy[0][i].numpy() #retrieves the bounding box points, confidence score, class,
            
            origin = (x0.astype(int), y0.astype(int));
            end = (x1.astype(int), y1.astype(int));
            
            #label = cv2.rectangle(frames, origin,end, (255,0,0), 2 )
            
            label = cv2.rectangle(frames, origin, end, (255,255,0), 2 )
            label = cv2.putText(frames, str(cla), (x0.astype(int), y0.astype(int) + 10), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 255, 255));
            
            
        
    if confi > confi_threshold:
        
        #If a new gesture is Present
        if (startTime == 0):
            scanning_class = cla
            gesturePresent = True;
            startTime = time.time();
        
        #if same class as before, start time
        if (scanning_class == cla and startTime > 0):
            elapseTime = round((time.time() - startTime), 2)
            label = cv2.putText(frames, str(elapseTime), (5, 30), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
            #if passes 3 three seconds, that gesture is added to the bank
            #print(elapseTime)
            if(elapseTime > delayDetection):    
                display_msg = "Gesture Recognised - " + str(cla);
                password.append(int(cla));
                print(display_msg)
                startTime = 0;
                    
    
    if (scanning_class != cla or confi < confi_threshold):
        gesturePresent = False;
        startTime = 0;
        #print('Bad Input. No Gesture Recognised or Not Clear');
            
    #if 3 inputs are detected        
    if (len(password) > 2):
        if password == masterPassword:
            print("You have Entered the Correct Password");
        else:
            print("Wrong Password");
        print(password);
        break
    
    #converts to a string 
    pass_text = ''.join(str(x) for x in password); 
    label = cv2.putText(frames, pass_text, (5, 500), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
    cv2.imshow('img', label)
    
    
    # Stop if escape key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release();
        cv2.destroyAllWindows();
        

        
        
        


        
cap.release();
cv2.destroyAllWindows();

#gets the time when the person entered. 
time_enter = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

saveFile = cv2.putText(img, time_enter, (5, 60), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
filename = 'savedImage.jpg'
cv2.imwrite(filename, saveFile)










No Face Detected; Timer Reset
Face is Detected
Gesture Recognised - 1.0
Gesture Recognised - 2.0
Gesture Recognised - 2.0
Wrong Password
[1, 2, 2]


True

In [297]:
cap.release();
cv2.destroyAllWindows();
